<a href="https://colab.research.google.com/github/HodaMemar/Patient-Similarity-through-Representation/blob/main/Create_Gold_Standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this section, a golden standard is produced to evaluate the quality of embedding vectors. The source of this gold standard is the registered final diagnoses for patients. In the structure of this golden standard, the method presented in the article https://doi.org/10.1016/j.artmed.2020.101900  is used. In this method, the order of recorded final diagnoses is used for weighting.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
gold=pd.read_csv('/content/drive/MyDrive/diagnoses.csv')
gold

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,3,145834,1,0389
1,3,145834,2,78559
2,3,145834,3,5849
3,3,145834,4,4275
4,3,145834,5,41071
...,...,...,...,...
2142,1474,140051,5,2800
2143,1474,140051,6,42731
2144,1474,140051,7,5789
2145,1474,140051,8,99662


In [ ]:
list_Subject_id=gold.HADM_ID.unique().tolist()

In [ ]:
import numpy as np
Weights = {}
for i in range(1,50):
    Weights[i] = 2 / (np.exp(i-1) + 1)

ls_W=[]
for i in range(len(gold)):
    seq =gold.loc[i].SEQ_NUM
    ls_W.append(Weights[seq])
gold['Weight']=pd.DataFrame(ls_W)

In [ ]:
def simIndex(a,b):
    x={}
    x=a
    y={}
    y=b
    #print(x,y)
    commonhadm = list(set(x.keys()).intersection(set(y.keys())))

    if (len(commonhadm) < 1):
            return 0.0
    aw,bw = 0.0,0.0
    for i in x:
        aw = aw +x[i]
    for j in y:
        bw = bw +y[j]
    sum = 0
    for c in commonhadm:
            w1 = x[c]

            w2 = y[c]
            #print(w1,w2)
            sum = sum + min(w1, w2)
    similarityIndex = sum / np.mean([aw, bw])
    return round(similarityIndex, 4)


In [ ]:
gold.head()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,Weight
0,3,145834,1,0389,1.000000
1,3,145834,2,78559,0.537883
2,3,145834,3,5849,0.238406
3,3,145834,4,4275,0.094852
4,3,145834,5,41071,0.035972


In [ ]:
import time
ls_residual=[]
score1=[]
i=0
for key_i in list_Subject_id[0:5] :
    df_a=gold[gold['HADM_ID']==key_i][['ICD9_CODE','Weight']]
    a = dict(zip(df_a.ICD9_CODE, df_a.Weight))
    start = time.time()

    i=0
    for key_j in list_Subject_id[0:5]:
            try:
                df_b=gold[gold['HADM_ID']==key_j][['ICD9_CODE','Weight']]
                b = dict(zip(df_b.ICD9_CODE, df_b.Weight))
                print(simIndex(a,b))
                i=i+1
                score1.append([key_i,key_j,simIndex(a,b)])
            except:
                print("Error!")




    stop = time.time()
    duration = stop-start


1.0
0.0563
0.5399
0.0069
0.0
0.0563
1.0
0.0
0.1239
0.5295
0.5399
0.0
1.0
0.0
0.0
0.0069
0.1239
0.0
1.0
0.001
0.0
0.5295
0.0
0.001
1.0


In [ ]:
gold_df=pd.DataFrame(score1,columns=['p1','p2','SimIndex'])
gold_df

,p1,p2,SimIndex
0,145834,145834,1.0000
1,145834,150750,0.0563
2,145834,176176,0.5399
3,145834,190539,0.0069
4,145834,155252,0.0000
5,150750,145834,0.0563
6,150750,150750,1.0000
7,150750,176176,0.0000
8,150750,190539,0.1239
9,150750,155252,0.5295
